In [1]:
from data.gaussianGridDataset import gaussianGridDataset
from Online_Kernel_GAN import Online_Kernel_GAN

# Let's make a gaussian 2d grid dataset

In [2]:
data_config = {
            'num_grid':5,
            'n_data':100,
            'sigma':0.05
        }
data_gaussiangrid = gaussianGridDataset(n=data_config['num_grid'], n_data=data_config['n_data'], sig=data_config['sigma'])

# Let's create and train the GAN on this dataset

We fix the gamma value as 0.5. We can observe that the GAN with the online kernel classifier learns a gaussian 2d grid distribution. However, the performance of this GAN is worse than that of GAN with kernel SVM. All images are saved in 'out_image/out_gaussian2d_online_kernel'.

In [3]:
gan_gaussian2d = Online_Kernel_GAN(gamma=0.5, gamma_ratio=1, num_epochs=1000, n_data=data_config['n_data'], data=data_gaussiangrid)
gan_gaussian2d.train_GAN()

100%|██████████| 25/25 [00:00<00:00, 36.59it/s]


spent time for epoch 0 is 0.6895027160644531s
epoch # : 0 gamma : 0.5 gen loss : 1.992035150527954


 20%|██        | 5/25 [00:00<00:00, 49.79it/s]

Total number of generated modes is  3
The number of high quality samples among 2500 samples is 39
The mode dictionary is Counter({12: 23, 7: 9, 13: 7})


100%|██████████| 25/25 [00:00<00:00, 48.60it/s]


...

spent time for epoch 998 is 0.5897209644317627s
epoch # : 998 gamma : 0.5 gen loss : 1.024125099182129


 20%|██        | 5/25 [00:00<00:00, 48.66it/s]

Total number of generated modes is  22
The number of high quality samples among 2500 samples is 238
The mode dictionary is Counter({5: 22, 16: 22, 2: 18, 15: 17, 8: 16, 6: 16, 7: 13, 10: 13, 21: 11, 17: 11, 13: 11, 12: 11, 20: 9, 18: 9, 3: 8, 4: 7, 1: 7, 11: 6, 23: 4, 0: 4, 22: 2, 24: 1})


100%|██████████| 25/25 [00:00<00:00, 48.38it/s]


spent time for epoch 999 is 0.522648811340332s
epoch # : 999 gamma : 0.5 gen loss : 0.981890082359314
Total number of generated modes is  23
The number of high quality samples among 2500 samples is 240
The mode dictionary is Counter({16: 22, 15: 20, 8: 19, 13: 17, 12: 16, 6: 16, 22: 15, 7: 13, 0: 11, 2: 10, 14: 10, 5: 9, 3: 9, 19: 9, 20: 7, 1: 6, 4: 6, 11: 6, 17: 5, 21: 5, 23: 4, 9: 4, 18: 1})


# Now let's play with the MNIST dataset

In [4]:
from torchvision import transforms, datasets

img_size = 28

def mnist_data():
    compose = transforms.Compose(
        [transforms.Resize(img_size),
         transforms.ToTensor(),
         transforms.Normalize((.5,), (.5,))
        ])
    out_dir = 'data'
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

data_mnist = mnist_data()

# Performance on MNIST

In this experiment, we use a gaussian kernel and fix the gamma value as 0.1. All images are saved in 'out_image/out_mnist_online_kernel'. There is an error when we try to change the kernel into polynomial kernel. We can see that this GAN cannot generate fake data with good quality. Before this experiment, We need to make sure that the online kernel classifier works well on MNIST dataset.

In [5]:
gan_mnist = Online_Kernel_GAN(kernel='gaussian', lr=0.0002, gamma=0.1, gamma_ratio=1.0, g_steps=1, num_epochs=200, batch_size=200, img_size=img_size, data=data_mnist, data_type='mnist', model_type='VGAN')
gan_mnist.train_GAN()

100%|██████████| 300/300 [00:20<00:00, 14.53it/s]


spent time for epoch 0 is 20.681440591812134s
epoch # : 0 gamma : 0.1 gen loss : 1.0011231899261475


100%|██████████| 300/300 [00:20<00:00, 14.77it/s]


spent time for epoch 1 is 20.356979608535767s
epoch # : 1 gamma : 0.1 gen loss : 1.0051140785217285


100%|██████████| 300/300 [00:20<00:00, 14.71it/s]


...

spent time for epoch 60 is 20.372053384780884s
epoch # : 60 gamma : 0.1 gen loss : 0.9950025677680969


100%|██████████| 300/300 [00:20<00:00, 14.78it/s]


spent time for epoch 61 is 20.305959224700928s
epoch # : 61 gamma : 0.1 gen loss : 0.9881147742271423


 31%|███       | 92/300 [00:06<00:13, 14.87it/s]


KeyboardInterrupt: 